# <span style="color:gold">**Análisis exploratorio de datos en un Modelo de Bloques (Parte 2)**</span>
***

### **Editado por: Kevin Alexander Gómez**
#### Contacto: kevinalexandr19@gmail.com | [Linkedin](https://www.linkedin.com/in/kevin-alexander-g%C3%B3mez-2b0263111/) | [Github](https://github.com/kevinalexandr19)
***

### **Descripción**

En este notebook, desarrollaremos un flujo de trabajo para el análisis exploratorio de datos usando un <span style="color:gold">modelo de bloques</span> dentro de Python.

Este Notebook es parte del proyecto [**Python para Geólogos**](https://github.com/kevinalexandr19/manual-python-geologia), y ha sido creado con la finalidad de facilitar el aprendizaje en Python para estudiantes y profesionales en el campo de la Geología.
***

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets  # Widgets de Jupyter
from tqdm import tqdm         # Barra de progreso en bucles

# Estilo de visualización
sns.set(style="ticks", context="talk")

# Número de decimales a mostrar en un DataFrame
pd.set_option("display.float_format", lambda x: f"{x:.5f}")

Cargaremos el archivo que generamos en la primera parte, llamado `modelo.csv`:

In [ ]:
modelo = pd.read_csv("modelo.csv")

In [ ]:
modelo.head()

Ahora que ya tenemos cargada la información dentro de Python, crearemos herramientas de visualización interactiva 2D y 3D para entender mejor el modelo.

# **4. Visualización de datos**

Antes de crear los gráficos, crearemos columnas que contengan los colores específicos de cada categoría en Cu, Mo y litología.\
Para realizar esta tarea, usaremos el método `map` que evalúa cada fila del DataFrame en base a los valores de un diccionario:

In [ ]:
print(modelo["LITO"].unique())
print(modelo["CU"].unique())
print(modelo["MO"].unique())

In [ ]:
color_lito = {"CUATERNARIO": "gray", "ESQUISTO": "maroon", "ANDESITA": "red", "DIQUES": "yellow", "PORFIDO": "blue", "INTRAMINERAL": "green"}
modelo["Color_LITO"] = modelo["LITO"].map(color_lito)

color_cu = {"Cu < 1.0 %": "blue", "1.0 <= Cu < 2.0 %": "yellow", "Cu >= 2.0 %": "red"}
modelo["Color_CU"] = modelo["CU"].map(color_cu)

color_mo = {"Mo < 200 ppm": "blue", "200 <= Mo < 400 ppm": "yellow", "Mo >= 400 ppm": "red"}
modelo["Color_MO"] = modelo["MO"].map(color_mo)

In [ ]:
modelo.head()

## **4.1. Visualización 2D del modelo de bloques**
Para visualizar el modelo de bloques, empezaremos con figuras en 2 dimensiones (planos y secciones).\
Empezaremos creando listas con los valores únicos de coordenadas X, Y, Z:

In [ ]:
# Creamos una lista por cada eje de coordenadas
z = list(modelo["Z"].unique())  # Elevación
y = list(modelo["Y"].unique())  # Norte
x = list(modelo["X"].unique())  # Este

In [ ]:
print(f"Primeras 10 coordenadas de Z: {z[:10]}")
print(f"Primeras 10 coordenadas de Y: {y[:10]}")
print(f"Primeras 10 coordenadas de X: {x[:10]}")

Ahora, generamos un resumen de las dimensiones del modelo (coordenadas X, Y, Z):

In [ ]:
# Resumen de la elevación (Z) del modelo
print(f"El modelo se encuentra entre los {min(z):,.0f} y {max(z):,.0f} metros de elevación.")
print(f"El modelo se eleva {max(z) - min(z):.0f} metros en dirección vertical.\n")

# Resumen de la coordenada Norte (Y)
print(f"El modelo se ubica entre los {min(y):,.0f} y {max(y):,.0f} metros en dirección Norte.")
print(f"El modelo se extiende {max(y) - min(y):.0f} metros en dirección Norte.\n")

# Resumen de la coordenada Este (N)
print(f"El modelo se ubica entre los {min(x):,.0f} y {max(x):,.0f} metros en dirección Este.")
print(f"El modelo se extiende {max(x) - min(x):.0f} metros en dirección Este.\n")

# Resumen de las dimensiones en bloques
print(f"El modelo está contenido en un volumen de {len(x)} x {len(y)} x {len(z)} bloques.")
print(f"Cada bloque mide {x[1] - x[0]} x {y[1] - y[0]} x {z[1] - z[0]} metros.")

Ahora, crearemos valores de referencia espacial para los gráficos:

In [ ]:
# Valores mínimos y máximos de los ejes
x_min, x_max = -100, 1500
y_min, y_max = -100, 1900
z_min, z_max = 0, 1400

In [ ]:
# Espacios lineales cada 200 metros
rango_x = np.arange(x_min + 100, x_max + 200, 200)
rango_y = np.arange(y_min + 100, y_max + 200, 200)
rango_z = np.arange(z_min, z_max + 200, 200)

Empezaremos creando un plano horizontal que pase por una coordenada Z específica, a la cual agregaremos el valor de ley de Cu.\
Primero, seleccionaremos aquellos bloques que pasen por el plano/sección usando la lista de coordenadas `x`, `y` y `z`:

In [ ]:
print(f"La coordenada X tiene {len(x)} bloques de extensión")
print(f"La coordenada Y tiene {len(y)} bloques de extensión")
print(f"La coordenada Z tiene {len(z)} bloques de extensión")

In [ ]:
# Asignamos un nivel horizontal para el eje Z
i = 50

# Asignamos el nombre de la columna a visualizar
col = "Color_CU"

# Seleccionamos aquella parte del modelo que tenga el nivel asignado
corte = modelo[modelo["Z"] == z[i]]

Los cortes que generemos solamente incluirán aquellos bloques que pasen por una coordenada X/Y/Z específica.\
En este caso, los bloques cortan el nivel $Z=600$:

In [ ]:
corte.head()

Usando los bloques de este DataFrame, podemos crear un plano/sección de corte del modelo:

In [ ]:
# Figura principal
fig, ax = plt.subplots(figsize=(12, 10), subplot_kw={"aspect": 1})

# Diagrama de dispersión
ax.scatter(corte["X"], corte["Y"], c=corte[col], marker="s", s=5)

# Estableciendo los ticks para X e Y
ax.set_xticks(rango_x)
ax.set_yticks(rango_y)

# Agregamos etiquetas a los ejes
ax.set_xlabel("Este (m)", fontsize=25)
ax.set_ylabel("Norte (m)", fontsize=25)

# Leyenda
for ley, color in color_cu.items():
    ax.scatter([], [], c=color, marker="s", s=100, label=ley)
ax.legend(loc=(1.03, 0.5), fontsize=20)

# Título
ax.set_title(f"Z = {z[i]:.0f} m", fontsize=30)

# Grilla
ax.grid(linewidth=0.4, color="black")
ax.set_axisbelow(False)

# Límites
ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)

plt.show()

Un solo plano/sección no es suficiente para entender la distribución espacial del modelo así que ahora crearemos una herramienta interactiva para visualizar varios cortes de manera rápida y eficiente. 

### **Visualización interactiva usando widgets de Jupyter**
Usaremos el código del gráfico anterior y la función `interact` para generar una visualización interactiva:

In [ ]:
# Definimos la función para graficar un corte paralelo al plano xy del modelo de bloques
def corte_xy(i, col):
    # Selección de bloques
    corte = modelo[modelo["Z"] == z[i]]

    # Figura principal
    fig, ax = plt.subplots(figsize=(15, 8), subplot_kw={"aspect": 1})
    
    # Crea un gráfico de acuerdo al tipo de columna seleccionada
    if col == "CU":
        ax.scatter(corte["X"], corte["Y"], c=corte["Color_CU"], marker="s", s=5)
        for ley, color in color_cu.items():
            ax.scatter([], [], c=color, marker="s", s=100, label=ley)
        ax.legend(loc=(1.03, 0.5), fontsize=20)
    elif col == "MO":
        ax.scatter(corte["X"], corte["Y"], c=corte["Color_MO"], marker="s", s=5)
        for ley, color in color_mo.items():
            ax.scatter([], [], c=color, marker="s", s=100, label=ley)
        ax.legend(loc=(1.03, 0.5), fontsize=20)
    elif col == "LITO":
        ax.scatter(corte["X"], corte["Y"], c=corte["Color_LITO"], marker="s", s=5)
        for ley, color in color_lito.items():
            ax.scatter([], [], c=color, marker="s", s=100, label=ley)
        ax.legend(loc=(1.03, 0.5), fontsize=20)

    # Establecemos los ticks
    ax.set_xticks(rango_x)
    ax.set_yticks(rango_y)

    # Grilla
    ax.grid(linewidth=0.3, color="black")  
    ax.set_axisbelow(False) # Coloca la grilla por encima de la figura
  
    # Límites de la figura
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)
    
    # Título
    ax.set_title(f"Elevación = {z[i]:.0f} m", fontsize=30)
    
    # Etiquetas de los ejes
    ax.set_xlabel("Este (m)", fontsize=24)
    ax.set_ylabel("Norte (m)", fontsize=24)
    
    # Ajuste de figura
    plt.subplots_adjust(bottom=0, right=1.0, top=0.8)
    
    plt.show()

widgets.interact(corte_xy, i=(10, len(z)-10, 10), col=["CU", "MO", "LITO"]);

In [ ]:
# Definimos la función para graficar un corte paralelo al plano xz del modelo de bloques
def corte_xz(i, col):
    # Selección de bloques
    corte = modelo[modelo["Y"] == y[i]]

    # Figura principal
    fig, ax = plt.subplots(figsize=(15, 8), subplot_kw={"aspect": 1})
    
    # Crea un gráfico de acuerdo al tipo de columna seleccionada
    if col == "CU":
        ax.scatter(corte["X"], corte["Z"], c=corte["Color_CU"], marker="s", s=10)
        for ley, color in color_cu.items():
            ax.scatter([], [], c=color, marker="s", s=100, label=ley)
        ax.legend(loc=(1.03, 0.5), fontsize=20)
    elif col == "MO":
        ax.scatter(corte["X"], corte["Z"], c=corte["Color_MO"], marker="s", s=10)
        for ley, color in color_mo.items():
            ax.scatter([], [], c=color, marker="s", s=100, label=ley)
        ax.legend(loc=(1.03, 0.5), fontsize=20)
    elif col == "LITO":
        ax.scatter(corte["X"], corte["Z"], c=corte["Color_LITO"], marker="s", s=10)
        for ley, color in color_lito.items():
            ax.scatter([], [], c=color, marker="s", s=100, label=ley)
        ax.legend(loc=(1.03, 0.5), fontsize=20)

    # Establecemos los ticks
    ax.set_xticks(rango_x)
    ax.set_yticks(rango_z)

    # Grilla
    ax.grid(linewidth=0.3, color="black")  
    ax.set_axisbelow(False) # Coloca la grilla por encima de la figura
  
    # Límites de la figura
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(z_min, z_max)
    
    # Título
    ax.set_title(f"Norte = {y[i]:.0f} m", fontsize=30)
    
    # Etiquetas de los ejes
    ax.set_xlabel("Este (m)", fontsize=24)
    ax.set_ylabel("Elevación (m)", fontsize=24)
    
    # Ajuste de figura
    plt.subplots_adjust(bottom=0, right=1.0, top=0.8)
    
    plt.show()

widgets.interact(corte_xz, i=(10, len(y)-10, 10), col=["CU", "MO", "LITO"]);

In [ ]:
# Definimos la función para graficar un corte paralelo al plano yz del modelo de bloques
def corte_yz(i, col):
    # Selección de bloques
    corte = modelo[modelo["X"] == x[i]]

    # Figura principal
    fig, ax = plt.subplots(figsize=(15, 8), subplot_kw={"aspect": 1})
    
    # Crea un gráfico de acuerdo al tipo de columna seleccionada
    if col == "CU":
        ax.scatter(corte["Y"], corte["Z"], c=corte["Color_CU"], marker="s", s=10)
        for ley, color in color_cu.items():
            ax.scatter([], [], c=color, marker="s", s=100, label=ley)
        ax.legend(loc=(1.03, 0.5), fontsize=20)
    elif col == "MO":
        ax.scatter(corte["Y"], corte["Z"], c=corte["Color_MO"], marker="s", s=10)
        for ley, color in color_mo.items():
            ax.scatter([], [], c=color, marker="s", s=100, label=ley)
        ax.legend(loc=(1.03, 0.5), fontsize=20)
    elif col == "LITO":
        ax.scatter(corte["Y"], corte["Z"], c=corte["Color_LITO"], marker="s", s=10)
        for ley, color in color_lito.items():
            ax.scatter([], [], c=color, marker="s", s=100, label=ley)
        ax.legend(loc=(1.03, 0.5), fontsize=20)

    # Establecemos los ticks
    ax.set_xticks(rango_y)
    ax.set_yticks(rango_z)

    # Grilla
    ax.grid(linewidth=0.3, color="black")  
    ax.set_axisbelow(False) # Coloca la grilla por encima de la figura
  
    # Límites de la figura
    ax.set_xlim(y_min, y_max)
    ax.set_ylim(z_min, z_max)
    
    # Título
    ax.set_title(f"Este = {x[i]:.0f} m", fontsize=30)
    
    # Etiquetas de los ejes
    ax.set_xlabel("Norte (m)", fontsize=24)
    ax.set_ylabel("Elevación (m)", fontsize=24)
    
    # Ajuste de figura
    plt.subplots_adjust(bottom=0, right=1.0, top=0.8)
    
    plt.show()

widgets.interact(corte_yz, i=(10, len(x)-10, 10), col=["CU", "MO", "LITO"]);

## **4.2. Visualización 3D del modelo de bloques**
Ahora que hemos verificado la distribución espacial del modelo en 2D, usaremos la librería `vpython` para observar en 3D el modelo de bloques.\
Primero, vamos a extraer aquellos bloques que forman la superficie del modelo, divididos de acuerdo a una categoría (litología, ley, etc.):
> La función `display` nos permitirá mostrar cualquier tipo de objeto dentro de Jupyter (incluyendo widgets de Jupyter).

In [ ]:
from vpython import *
from IPython.display import display

In [ ]:
def extract_surface(model, feature, cellsize=10):
    print(f"Extracción de bloques superficiales para un modelo de {len(model)} bloques")
    
    features = list(model[feature].unique())
    blocks = []
    
    for name in features:
        df = model[model[feature] == name]
        zmax = df["Z"].max()
        ymax = df["Y"].max()
        xmax = df["X"].max()

        zmin = df["Z"].min()
        ymin = df["Y"].min()
        xmin = df["X"].min()

        zcells = int((zmax - zmin)/cellsize)
        ycells = int((ymax - ymin)/cellsize)
        xcells = int((xmax - xmin)/cellsize)
        
        for x in tqdm(range(0, xcells + 1)):
            yz = df[df["X"] == (xmin + cellsize*x)]
            for y in range(0, ycells + 1):
                z = yz[yz["Y"] == (ymin + cellsize*y)]
                if len(z) != 0:
                    block1 = z[z["Z"] == z["Z"].max()].values[0]
                    block2 = z[z["Z"] == z["Z"].min()].values[0]
                    blocks.append(list(block1))
                    blocks.append(list(block2))
        
        for z in tqdm(range(0, zcells + 1)):
            xy = df[df["Z"] == (zmin + cellsize*z)]
            for y in range(0, ycells + 1):
                x = xy[xy["Y"] == (ymin + cellsize*y)]
                if len(x) != 0:
                    block1 = x[x["X"] == x["X"].max()].values[0]
                    block2 = x[x["X"] == x["X"].min()].values[0]
                    blocks.append(list(block1))
                    blocks.append(list(block2))
        
        for z in tqdm(range(0, zcells + 1)):
            xy = df[df["Z"] == (zmin + cellsize*z)]
            for x in range(0, xcells + 1):
                y = xy[xy["X"] == (xmin + cellsize*x)]
                if len(y) != 0:
                    block1 = y[y["Y"] == y["Y"].max()].values[0]
                    block2 = y[y["Y"] == y["Y"].min()].values[0]
                    blocks.append(list(block1))
                    blocks.append(list(block2))
    
    # DataFrame con los bloques seleccionados
    blocks = pd.DataFrame(blocks, columns=model.columns).drop_duplicates()
    
    # Final
    print(f"Proceso finalizado, se extrajeron {len(blocks)} bloques superficiales.")
    print(f"Reducción del {(len(model) - len(blocks))/len(model):.1%} del modelo original.")
    
    return blocks

Usando la función `surface`, extraemos los bloques superficiales del modelo, separados de acuerdo a la litología:

In [ ]:
surface = extract_surface(modelo, "LITO")

Reducimos la cantidad de bloques a visualizar en aproximadamente un 84%:

In [ ]:
surface

Para agregar color a un modelo de bloques usando `vpython`, debemos transformar los colores de la categoría a RGB normalizado:
> RGB normalizado se representa por una tupla de 3 valores numéricos (red, green blue), cuya suma es igual a 1.

In [ ]:
from matplotlib import colors

surface["Color"] = surface["Color_LITO"].apply(colors.to_rgb)

In [ ]:
surface.head()

Y ahora, graficamos el modelo 3D:

In [ ]:
# 1. MODELO 3D
# Origen en el centro del modelo
scene = canvas(center=vector(max(x)/2, max(z)/2, max(y)/2))

# Diccionario que almacenará cada sólido y su respectivo nombre
solidos = dict()

# Bucle para graficar cada sólido del modelo
for lito in surface["LITO"].unique():
    df = surface[surface["LITO"] == lito]
    blocks = df[["X", "Z", "Y", "Color"]].to_numpy()
    
    boxes = []
    for block in blocks:
        b = box(color=vector(*block[3]),
                pos=vector(*block[:3]),
                size=vector(10, 10, 10))
        boxes.append(b)
    
    volume = compound(boxes)
    solidos[lito] = volume

# Iluminación del modelo 3D
distant_light(direction=vector(1, 1, 1), color=vector(0.5, 0.5, 0.5))
distant_light(direction=vector(1, 1, -1), color=vector(0.5, 0.5, 0.5))


# 2. WIDGET PARA OCULTAR LOS SÓLIDOS DEL MODELO
names = []
checkbox_objects = []
for lito in surface["LITO"].unique():
    checkbox_objects.append(widgets.Checkbox(value=True, description=lito))
    names.append(lito)

# Diccionario con el nombre de cada sólido y su respectivo widget
checkbox_dict = {names[i]: checkbox for i, checkbox in enumerate(checkbox_objects)}

# Interfaz de usuario (Caja de 3 filas x 2 columnas)
ui = widgets.HBox([widgets.VBox(children=checkbox_objects[:3]),
                   widgets.VBox(children=checkbox_objects[3:])])

# Funcionalidad del widget
def select_data(**kwargs):
    for key in kwargs:
        if kwargs[key] is True:
            solidos[key].visible = True
        else:
            solidos[key].visible = False

# Interacción entre la función y el diccionario a través de un output
out = widgets.interactive_output(select_data, checkbox_dict)
display(ui, out)

***